In [10]:
import cv2 as cv
import os

In [11]:
camera = cv.VideoCapture(0)
if not camera.isOpened():
    print("Camera not opened")
    exit()  

In [12]:
Labels = ["angry", "happy", "sad", "surprise", "neutral"]

for label in Labels:
    if not os.path.exists(label):
        os.mkdir(label)
        

In [13]:
import cv2 as cv

for folder in Labels:
    #using count variable to name the images in the dataset.
    count = 0
    #Taking input to start the capturing
    print("Press 's' to start data collection for "+folder)
    userinput = input()
    if userinput != 's':
        print("Wrong Input..........")
        exit()
    #clicking 200 images per label, you could change as you want.    
    while count<400:
        #read returns two values one is the exit code and other is the frame
        status, frame = camera.read()
        #check if we get the frame or not
        if not status:
            print("Frame is not been captured..Exiting...")
            break
        #convert the image into gray format for fast caculation
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        #display window with gray image
        cv.imshow("Video Window",gray)

        #using cv to detect faces in the frame and croppe it. this is what is going to be stored in the dataset
        face_cascade = cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for x,y,w,h in faces:
            cv.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 1)
            roi_gray = gray[y:y+h, x:x+w]
            cropped_img = cv.resize(roi_gray,(48,48))
            cv.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv.NORM_L2, dtype=cv.CV_32F)
            cv.imwrite(folder+'/'+ folder +str(count)+'.jpg',cropped_img)
            count+=1
        #display the frame with rectangle around the face
        cv.imshow("Video Window",frame)
        count+=1

        #to quite the display window press 'q'
        if cv.waitKey(1) & 0xFF == ord('q'):
            break
    

# When everything done, release the capture
camera.release()
cv.destroyAllWindows()

Press 's' to start data collection for angry
Press 's' to start data collection for happy
Press 's' to start data collection for sad
Press 's' to start data collection for surprise
Press 's' to start data collection for neutral


In [ ]:
#Todo esto es parte de data_transformation.py

import pandas as pd
import numpy as np
import os
import cv2

# Definir las emociones y sus correspondientes valores enteros
emotion_dict = {0: "angry", 1: "happy", 2: "sad", 3: "surprise", 4: "neutral"}

# Crear listas vacías para almacenar los datos del conjunto de datos
emotion = []
pixels = []
usage = []

# Iterar a través de cada carpeta de emociones
for emotion_folder in os.listdir("dataset"):
    # Obtener el valor entero correspondiente a la emoción actual
    emotion_label = list(emotion_dict.keys())[
        list(emotion_dict.values()).index(emotion_folder)
    ]

    # Iterar a través de cada imagen en la carpeta actual
    for image_filename in os.listdir(os.path.join("dataset", emotion_folder)):
        # Leer la imagen y convertirla a escala de grises
        image_path = os.path.join("dataset", emotion_folder, image_filename)
        image = cv2.imread(image_path)
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # Aplanar la imagen en un vector de una dimensión
        flat_image = gray_image.flatten()

        # Agregar los datos al conjunto de datos
        emotion.append(emotion_label)
        flat_image_to_string = " ".join(str(x) for x in list(flat_image))
        pixels.append(flat_image_to_string)

        # Establecer la etiqueta de uso del conjunto de datos (Training, PrivateTest, PublicTest)
        rand_num = np.random.rand()
        if rand_num < 0.5:
            usage.append("Training")
        elif rand_num < 0.75:
            usage.append("PrivateTest")
        else:
            usage.append("PublicTest")

# Crear un dataframe de pandas para almacenar los datos del conjunto de datos
df = pd.DataFrame({"emotion": emotion, "pixels": pixels, "Usage": usage})

# Guardar el conjunto de datos como un archivo CSV
df.to_csv("dataset.csv", index=False)
